In [1]:
import json
import pandas as pd
from pprint import pprint
import glob
import re
from functools import reduce
import numpy as np
import os

In [2]:
with open('/Users/tommaso/Desktop/Biathlon_Azzurro/garfa_proj/data/keys_men.json', mode = 'r') as f:

    keys_men = json.load(f)

with open('/Users/tommaso/Desktop/Biathlon_Azzurro/garfa_proj/data/keys_women.json', mode = 'r') as f:

    keys_women = json.load(f) 

In [2]:
list_keys_w = list(keys_women.keys())
keys_women = {list_keys_w[i]:int(keys_women[list_keys_w[i]])  for i in range(len(keys_women))\
    if '/Users' not in list_keys_w[i]}

NameError: name 'keys_women' is not defined

In [ ]:
list_keys_m = list(keys_men.keys())
keys_men = {list_keys_m[i]:int(keys_men[list_keys_m[i]])  for i in range(len(keys_men))}

In [5]:
tmp = pd.read_csv('~/Desktop/biathlon_analysis/biathlon_data/men/2016-2017_m/Anterselva/loop_1_Anterselva_individual.txt', sep = '\t')

In [6]:
tmp.head()

,Rank,Bib,Family Name,Given Name,Nation,Cumulative Time,Ski Time,Loop Time,Course Time,Range Time,Shooting Time,Penalty Time
0,1,33,Shipulin,Anton,RUS,+8.6,+8.6,+8.6,+17.4,43.0,25.0,NaN
1,2,28,Fourcade,Martin,FRA,9:38.5,9:38.5,9:38.5,8:46.7,8.8,8.0,NaN
2,3,16,Semenov,Sergii,UKR,+16.0,+16.0,+16.0,+18.5,6.3,3.0,NaN
3,4,32,Bjoerndalen,Ole Einar,NOR,+19.3,+19.3,+19.3,+23.6,4.5,3.0,NaN
4,5,30,Schempp,Simon,GER,+12.7,+12.7,+12.7,+14.8,6.7,6.0,NaN


In [7]:
tmp['race_type'] = ['Individual']*len(tmp)
tmp['year'] = ['2016-2017']*len(tmp)
tmp['location'] = ['Anterselva']*len(tmp)

In [8]:
tmp_2 = tmp['race_type'][0] + ' ' + tmp['year'][0] + ' ' + tmp['location'][0] + ' '+  tmp['Given Name'][0] \
  + ' '+  tmp['Given Name'][1]  

In [3]:
path = '/Users/tommaso/Desktop/biathlon_analysis/biathlon_data/'

# sex should be either men or women
def year_venue(sex):
    years = []
    for item in glob.glob( path + sex + '/*'):
        years.append(item.split('/')[-1])

    venues = []
    for year in years:
        files = glob.glob( path + sex + '/' + year + '/*')
        venues_year = list(map(lambda x : x.split('/')[-1], files))
        venues = list(set(venues + venues_year))
    
    return (years, venues)

In [3]:
years_men, venues_men = year_venue('men')
years_women, venues_women = year_venue('women')

NameError: name 'year_venue' is not defined

In [2]:
race_types = {'sprint': 'Sprint', 'sprint_2': 'Sprint', 'individual': 'Individual',\
     'individual_2': 'Individual', 'mass_start': 'Mass start', 'mass_start_2': 'Mass start',\
    'pursuit': 'Pursuit', 'pursuit_2': 'Pursuit'} 

In [12]:
list(race_types.keys())

['sprint',
 'sprint_2',
 'individual',
 'individual_2',
 'mass_start',
 'mass_start_2',
 'pursuit',
 'pursuit_2']

In [13]:
def function(x):
    return x

In [14]:
reduce(lambda x, y: x + '(?:_2)*' + '|' + y , list(race_types.keys())) + '(?:_2)*'

'sprint(?:_2)*|sprint_2(?:_2)*|individual(?:_2)*|individual_2(?:_2)*|mass_start(?:_2)*|mass_start_2(?:_2)*|pursuit(?:_2)*|pursuit_2(?:_2)*'

In [15]:
pattern = 'sprint(?:_2)*'
re.findall(pattern, 'sprint')

['sprint']

In [16]:
glob.glob( path + 'men' + '/' + years_men[0] + f'/{venues_men[1]}/*')

[]

In [17]:
path + 'men' + '/' + years_men[0] + f'/{venues_men[0]}'

'/Users/tommaso/Desktop/biathlon_analysis/biathlon_data/men/2014-2015_m/Khanty-Mansiysk'

In [23]:
with open(glob.glob( path + 'men' + '/' + years_men[0] + f'/{venues_men[0]}/*')[2], mode = 'r') as f:
    header = f.readlines()[0][:-1].split('\t')

In [7]:
df_loop = pd.DataFrame(columns = header)
for year in years_men:
    venues = list(map(lambda x: x.split('/')[-1],glob.glob( path + 'men' + '/' + year + '/*')))
    for venue in venues:
        if not os.path.exists(path + 'men/' + year + '/' + venue + '/intermediates'):
            os.mkdir(path + 'men/' + year + '/' + venue + '/intermediates')
        files_list = glob.glob( path + 'men' + '/' + year + f'/{venue}/*')
        bool_mask = list(map(lambda x: len(re.findall('loop_[0-9]', x)) > 0, files_list))
        loop_files = np.array(files_list)[np.array(bool_mask)]
        for loop_file in loop_files:            
            df = pd.read_csv(loop_file, sep = '\t')
            pattern = reduce(lambda x, y: x + '(?:_2)*|' + y , list(race_types.keys())) + '(?:_2)*'
            race_type = re.findall(pattern, loop_file)[0]
            df['race_type'] = [race_types[race_type]]*len(df)
            if not os.path.exists(path + 'men/' + year + '/' + venue + f'/intermediates/{race_type}'):
                os.mkdir(path + 'men/' + year + '/' + venue + f'/intermediates/{race_type}')
            df['year'] = [year[:-2]]*len(df)
            df['location'] = [venue]*len(df)
            df['loop'] = [re.findall('loop_[0-9]', loop_file)[0]]*len(df)
            key = df['race_type'][0] + ' ' + df['year'][0] + ' ' + df['location'][0] + ' '+  df['Given Name'][0] \
                + ' '+  df['Given Name'][1]  
            if key in list(keys_men.keys()):
                df['race_key'] = [keys_men[key]] * len(df)               
                df_loop = pd.concat([df, df_loop])

NameError: name 'header' is not defined

In [25]:
df_loop.reset_index(inplace = True, drop= True)
df_loop.race_key = df_loop.race_key.astype(int) 
df_loop['full_name'] = df_loop['Given Name'] + ' ' + df_loop['Family\xa0Name'] 
df_loop.full_name[df_loop.full_name == 'Alexandr Loginov'] = 'Alexander Loginov'
df_loop 

<ipython-input-25-32f6065f0070>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loop.full_name[df_loop.full_name == 'Alexandr Loginov'] = 'Alexander Loginov'


,Rank,Bib,Family Name,Given Name,Nation,Cumulative Time,Loop Time,Course Time,Range Time,Shooting Time,Penalty Time,race_type,year,location,loop,race_key,Ski Time,,"""30 min.",full_name
0,1.0,3.0,Fourcade,Martin,FRA,+27.2,+11.3,+1.4,+22.5,10.0,+22.2,Pursuit,2015-2016,Presque_Isle,loop_1,44,NaN,NaN,NaN,Martin Fourcade
1,2.0,1.0,Boe,Johannes Thingnes,NOR,6:18.1,+13.1,4:56.6,+24.1,12.0,+23.8,Pursuit,2015-2016,Presque_Isle,loop_1,44,NaN,NaN,NaN,Johannes Thingnes Boe
2,3.0,2.0,Shipulin,Anton,RUS,+32.3,+17.4,+3.1,+27.1,14.0,+22.0,Pursuit,2015-2016,Presque_Isle,loop_1,44,NaN,NaN,NaN,Anton Shipulin
3,4.0,11.0,Eder,Simon,AUT,+1:09.2,+13.3,+11.8,+13.5,21.0,+22.8,Pursuit,2015-2016,Presque_Isle,loop_1,44,NaN,NaN,NaN,Simon Eder
4,5.0,10.0,Bjoentegaard,Erlend,NOR,+1:02.6,+8.7,+13.1,+30.0,17.0,+0.4,Pursuit,2015-2016,Presque_Isle,loop_1,44,NaN,NaN,NaN,Erlend Bjoentegaard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56723,95.0,66.0,Puzulis,Rolands,LAT,+1:26.7,+1:26.7,+37.3,21.9,21.0,+48.2,Sprint,2014-2015,Oberhof,loop_1,15,NaN,NaN,NaN,Rolands Puzulis
56724,96.0,81.0,Oblak,Lenart,SLO,+42.4,+42.4,+48.9,13.5,14.0,+0.7,Sprint,2014-2015,Oberhof,loop_1,15,NaN,NaN,NaN,Lenart Oblak
56725,97.0,86.0,Guzik,Grzegorz,POL,+3:27.8,+3:27.8,+59.0,48.9,49.0,+2:00.6,Sprint,2014-2015,Oberhof,loop_1,15,NaN,NaN,NaN,Grzegorz Guzik
56726,98.0,67.0,Almoukov,Alexei,AUS,+2:27.2,+2:27.2,+41.1,24.9,23.0,+1:41.9,Sprint,2014-2015,Oberhof,loop_1,15,NaN,NaN,NaN,Alexei Almoukov


In [ ]:
with open(glob.glob( path + 'women' + '/' + years_women[0] + f'/{venues_women[1]}/*')[1], mode = 'r') as f:
    header = f.readlines()[0][:-1].split('\t')

In [6]:
df_loop_women = pd.DataFrame(columns = header)
for year in years_women:
    venues = list(map(lambda x: x.split('/')[-1],glob.glob( path + 'women' + '/' + year + '/*')))
    for venue in venues:
        if not os.path.exists(path + 'women/' + year + '/' + venue + '/intermediates'):
            os.mkdir(path + 'women/' + year + '/' + venue + '/intermediates')
        files_list = glob.glob( path + 'women' + '/' + year + f'/{venue}/*')
        bool_mask = list(map(lambda x: len(re.findall('loop_[0-9]', x)) > 0, files_list))
        loop_files = np.array(files_list)[np.array(bool_mask)]
        for loop_file in loop_files:       
            df = pd.read_csv(loop_file, sep = '\t')
            pattern = reduce(lambda x, y: x + '(?:_2)*|' + y , list(race_types.keys())) + '(?:_2)*'
            race_type = re.findall(pattern, loop_file)[0]
            df['race_type'] = [race_types[race_type]]*len(df)
            if not os.path.exists(path + 'women/' + year + '/' + venue + f'/intermediates/{race_type}'):
                os.mkdir(path + 'women/' + year + '/' + venue + f'/intermediates/{race_type}')
            df['year'] = [year[:-2]]*len(df)
            df['location'] = [venue]*len(df)
            df['loop'] = [re.findall('loop_[0-9]', loop_file)[0]]*len(df)
            key = df['race_type'][0] + ' ' + df['year'][0] + ' ' + df['location'][0] + ' '+  df['Given Name'][0] \
                + ' '+  df['Given Name'][1]  
            if key in list(keys_women.keys()):
                df['race_key'] = [keys_women[key]] * len(df)               
                df_loop_women = pd.concat([df, df_loop_women])

NameError: name 'header' is not defined

In [ ]:
df_loop_women.reset_index(inplace = True, drop= True)
df_loop_women.race_key = df_loop_women.race_key.astype(int) 
df_loop_women['full_name'] = df_loop_women['Given Name'] + ' ' + df_loop_women['Family\xa0Name'] 
df_loop_women.head()

,Rank,Bib,Family Name,Given Name,Nation,Cumulative Time,Loop Time,Course Time,Range Time,Shooting Time,Penalty Time,race_type,year,location,loop,race_key,Ski Time,full_name
0,1,18,Eckhoff,Tiril,NOR,+5.9,+5.9,+2.9,8.7,9.7,+0.9,Sprint,2020-2021,Pokljuka,loop_1,172,NaN,Tiril Eckhoff
1,2,23,Chevalier-Bouchet,Anais,FRA,7:18.9,7:18.9,6:21.9,5.1,4.3,+1.5,Sprint,2020-2021,Pokljuka,loop_1,172,NaN,Anais Chevalier-Bouchet
2,3,24,Sola,Hanna,BLR,+7.4,+7.4,+5.7,8.1,10.4,+0.2,Sprint,2020-2021,Pokljuka,loop_1,172,NaN,Hanna Sola
3,4,7,Herrmann,Denise,GER,+0.6,+0.6,+0.2,4.6,7.5,+2.4,Sprint,2020-2021,Pokljuka,loop_1,172,NaN,Denise Herrmann
4,5,35,Vittozzi,Lisa,ITA,+7.5,+7.5,+10.7,2.3,2.3,+1.1,Sprint,2020-2021,Pokljuka,loop_1,172,NaN,Lisa Vittozzi


In [3]:
dict_years_men = {'2016-2017':'2016-2017_m', '2017-2018':'2017-2018_m', '2018-2019':'2018-2019_m', '2019-2020':'2019-2020_m',\
     '2020-2021':'2020-2021_m', '2021-2022':'2021-2022_m'}

dict_years_women = {'2016-2017':'2016-2017_w', '2017-2018':'2017-2018_w', '2018-2019':'2018-2019_w', '2019-2020':'2019-2020_w',\
     '2020-2021':'2020-2021_w', '2021-2022':'2021-2022_w'}

In [4]:
def write_intermediates():
    sex = input('Wrute the sex [men/women]: ')
    year = input('Write the year: ')
    venue = input('Write the venue: ')
    race_type = input('Write the race type: ')
    while True:
        path = '/Users/tommaso/Desktop/biathlon_analysis/biathlon_data/'
        
        path += f'{sex}/'
        
        if sex == 'men':
            path += f'{dict_years_men[year]}/'
        elif sex == 'women':
            path += f'{dict_years_women[year]}/'     
        
        
        path+= f'{venue}/intermediates/'
        
        path += f'{race_type}'
        if os.path.exists(path):
            distance = input('Write all the distances: ')
            
        else:
            raise FileNotFoundError('The path does not exists')
        
        
        pattern_km = '[0-9][0-9]*\.[0-9]'
        
        for pat in re.findall(pattern_km, distance.replace(' ', '')):
            
            path_km = path + f'/{pat}km.txt'
            #print(path_km)
            if not os.path.exists(path_km):
                #print(path_km + '\n')
                with open('/Users/tommaso/Desktop/distances.txt', mode = 'a') as f:
                    
                    f.write('nano ' + path_km + '\n')
        cont_condition = input('You wanna continue?:[y/n]')
        if cont_condition == 'y':
            sex_change = input('You wanna change the sex?[y/n] ')
            if sex_change == 'y':
                sex = input('Write the sex: ')
            
            year_change = input('You wanna change the year?[y/n] ')
            if year_change == 'y':
                year = input('Write the year: ')
            
            venue_change = input('You wanna change the venue?[y/n] ')
            if venue_change == 'y':
                venue = input('Write the venue: ')
            
            race_change = input('You wanna change the race?[y/n] ')
            if race_change == 'y':
                race_type = input('Write the race: ')
            

        else:
            break 

In [9]:
write_intermediates()

In [16]:
pattern = '[0-9][0-9]*\.[0-9]'
tmp = re.findall(pattern, '1.0→ S1Shooting 1km 3.0→ S2Shooting 2km 5.0→ S3Shooting 3km 7.0→ S4Shooting 4km 9.0FinishCourse Time 1Course Time 2Course Time 3Course Time 4Course Time 5Course Total TimeLap Time 1Lap Time 2Lap Time 3Lap Time 4Lap Time 5Pursuit TimeRange Time 1Range Time 2Range Time'.replace(' ', ''))


In [20]:
np.array(tmp) 


array(['1.0', '3.0', '5.0', '7.0', '9.0'], dtype='<U3')